In [1]:
import pandas as pd

In [2]:
# Load the CSV file, skipping the first two rows and using the third row as column names
download_list = pd.read_csv('Data/ScreamingDataLink.csv', skiprows=1, header=1)

# Display the loaded data
download_list

,# YTID,start_seconds,end_seconds,positive_labels,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,---1_cCGK4M,0,10,"""/m/01g50p",/m/0284vy3,/m/06d_3,/m/07jdr,"/m/07rwm0c""",NaN,NaN,NaN
1,-20uudT97E0,30,40,"""/m/03qc9zr","/m/09x0r""",NaN,NaN,NaN,NaN,NaN,NaN
2,-2yygHLdpXc,20,30,"""/m/03qc9zr""",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-3bGlOhRkAo,140,150,"""/m/03qc9zr""",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-4pUrlMafww,1,11,"""/m/03qc9zr","/m/09x0r""",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1117,zhAUkNhQ7uM,10,20,"""/m/03qc9zr","/m/07s2xch""",NaN,NaN,NaN,NaN,NaN,NaN
1118,zkWoni28n64,70,80,"""/m/03qc9zr","/m/09x0r""",NaN,NaN,NaN,NaN,NaN,NaN
1119,zo10roED5Qg,70,80,"""/m/03qc9zr",/m/04rlf,"/m/09x0r""",NaN,NaN,NaN,NaN,NaN
1120,zodTMCJFKv8,110,120,"""/m/03qc9zr""",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
download_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1122 entries, 0 to 1121
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   # YTID           1122 non-null   object
 1   start_seconds    1122 non-null   int64 
 2   end_seconds      1122 non-null   int64 
 3   positive_labels  1122 non-null   object
 4   Unnamed: 4       846 non-null    object
 5   Unnamed: 5       379 non-null    object
 6   Unnamed: 6       138 non-null    object
 7   Unnamed: 7       43 non-null     object
 8   Unnamed: 8       6 non-null      object
 9   Unnamed: 9       2 non-null      object
 10  Unnamed: 10      1 non-null      object
dtypes: int64(2), object(9)
memory usage: 96.6+ KB


In [4]:
# Drop rows where the value in the '# YTID' column is equal to "#NAME?"
download_list = download_list[download_list['# YTID'] != '#NAME?']

# Reset the index of the DataFrame after dropping rows
download_list.reset_index(drop=True, inplace=True)

# Display the updated DataFrame
download_list

,# YTID,start_seconds,end_seconds,positive_labels,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,---1_cCGK4M,0,10,"""/m/01g50p",/m/0284vy3,/m/06d_3,/m/07jdr,"/m/07rwm0c""",NaN,NaN,NaN
1,-20uudT97E0,30,40,"""/m/03qc9zr","/m/09x0r""",NaN,NaN,NaN,NaN,NaN,NaN
2,-2yygHLdpXc,20,30,"""/m/03qc9zr""",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-3bGlOhRkAo,140,150,"""/m/03qc9zr""",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-4pUrlMafww,1,11,"""/m/03qc9zr","/m/09x0r""",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1099,zhAUkNhQ7uM,10,20,"""/m/03qc9zr","/m/07s2xch""",NaN,NaN,NaN,NaN,NaN,NaN
1100,zkWoni28n64,70,80,"""/m/03qc9zr","/m/09x0r""",NaN,NaN,NaN,NaN,NaN,NaN
1101,zo10roED5Qg,70,80,"""/m/03qc9zr",/m/04rlf,"/m/09x0r""",NaN,NaN,NaN,NaN,NaN
1102,zodTMCJFKv8,110,120,"""/m/03qc9zr""",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Function to trim and resample audio using FFmpeg
def trim_and_resample_audio(input_file, output_file, start_time, end_time, target_sample_rate):
    command = (
        f'ffmpeg -loglevel quiet -i "{input_file}" -ss {start_time} -to {end_time} -ar {target_sample_rate} "{output_file}"'
    )
    os.system(command)

In [8]:
import os
import yt_dlp

# Create a directory to store downloaded audio files if it doesn't exist
output_dir = 'Data/Screaming'
os.makedirs(output_dir, exist_ok=True)

# Create a yt-dlp instance
ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'wav',
    }],
    'outtmpl': os.path.join(output_dir, '%(id)s.%(ext)s'),
}

ydl = yt_dlp.YoutubeDL(ydl_opts)

# Loop through each row in the DataFrame and download the audio
for index, row in download_list.iterrows():
    yt_id = row['# YTID']
    start_seconds = row['start_seconds']
    end_seconds = row['end_seconds']

    # Construct the URL with time offsets
    url = f'https://www.youtube.com/watch?v={yt_id}'

    try:
        # Download the audio
        ydl.download([url])
        print(f"Downloaded audio for YTID {yt_id}")

        # Rename the downloaded file to match the YTID
        downloaded_file = os.path.join(output_dir, f"{yt_id}.wav")
        trimmed_file = os.path.join(output_dir, f"{yt_id}_out.wav")

        # Trim the audio using FFmpeg
        trim_and_resample_audio(downloaded_file, trimmed_file, start_seconds, end_seconds, 44100)

        # Remove the original downloaded file
        os.remove(downloaded_file)

        print(f"Trimmed audio for YTID {yt_id}")
    except Exception as e:
        print(f"Error downloading audio for YTID {yt_id}: {str(e)}")

print("Download and trimming completed.")

[youtube] Extracting URL: https://www.youtube.com/watch?v=---1_cCGK4M
[youtube] ---1_cCGK4M: Downloading webpage
[youtube] ---1_cCGK4M: Downloading ios player API JSON
[youtube] ---1_cCGK4M: Downloading android player API JSON
[youtube] ---1_cCGK4M: Downloading m3u8 information
[info] ---1_cCGK4M: Downloading 1 format(s): 140
[download] Destination: Data\Screaming\---1_cCGK4M.m4a
[download] 100% of  374.31KiB in 00:00:00 at 8.70MiB/s     
[FixupM4a] Correcting container of "Data\Screaming\---1_cCGK4M.m4a"
[ExtractAudio] Destination: Data\Screaming\---1_cCGK4M.wav
Deleting original file Data\Screaming\---1_cCGK4M.m4a (pass -k to keep)
Downloaded audio for YTID ---1_cCGK4M
Trimmed audio for YTID ---1_cCGK4M
[youtube] Extracting URL: https://www.youtube.com/watch?v=-20uudT97E0
[youtube] -20uudT97E0: Downloading webpage
[youtube] -20uudT97E0: Downloading ios player API JSON
[youtube] -20uudT97E0: Downloading android player API JSON
[youtube] -20uudT97E0: Downloading m3u8 information
[info]

ERROR: [youtube] -7gojlG6bE4: Video unavailable


Error downloading audio for YTID -7gojlG6bE4: ERROR: [youtube] -7gojlG6bE4: Video unavailable
[youtube] Extracting URL: https://www.youtube.com/watch?v=0-RnTXpp8Q0
[youtube] 0-RnTXpp8Q0: Downloading webpage
[youtube] 0-RnTXpp8Q0: Downloading ios player API JSON
[youtube] 0-RnTXpp8Q0: Downloading android player API JSON
[youtube] 0-RnTXpp8Q0: Downloading m3u8 information
[info] 0-RnTXpp8Q0: Downloading 1 format(s): 140
[download] Destination: Data\Screaming\0-RnTXpp8Q0.m4a
[download] 100% of    8.26MiB in 00:00:00 at 20.30MiB/s    
[FixupM4a] Correcting container of "Data\Screaming\0-RnTXpp8Q0.m4a"
[ExtractAudio] Destination: Data\Screaming\0-RnTXpp8Q0.wav
Deleting original file Data\Screaming\0-RnTXpp8Q0.m4a (pass -k to keep)
Downloaded audio for YTID 0-RnTXpp8Q0
Trimmed audio for YTID 0-RnTXpp8Q0
[youtube] Extracting URL: https://www.youtube.com/watch?v=09YQukdYVI4
[youtube] 09YQukdYVI4: Downloading webpage
[youtube] 09YQukdYVI4: Downloading ios player API JSON
[youtube] 09YQukdYVI4: 

ERROR: [youtube] 09YQukdYVI4: Private video. Sign in if you've been granted access to this video


Error downloading audio for YTID 09YQukdYVI4: ERROR: [youtube] 09YQukdYVI4: Private video. Sign in if you've been granted access to this video
[youtube] Extracting URL: https://www.youtube.com/watch?v=0A-jRMtiSGI
[youtube] 0A-jRMtiSGI: Downloading webpage
[youtube] 0A-jRMtiSGI: Downloading ios player API JSON
[youtube] 0A-jRMtiSGI: Downloading android player API JSON
[youtube] 0A-jRMtiSGI: Downloading m3u8 information
[info] 0A-jRMtiSGI: Downloading 1 format(s): 251
[download] Destination: Data\Screaming\0A-jRMtiSGI.webm
[download] 100% of  603.28KiB in 00:00:00 at 10.71MiB/s    
[ExtractAudio] Destination: Data\Screaming\0A-jRMtiSGI.wav
Deleting original file Data\Screaming\0A-jRMtiSGI.webm (pass -k to keep)
Downloaded audio for YTID 0A-jRMtiSGI
Trimmed audio for YTID 0A-jRMtiSGI
[youtube] Extracting URL: https://www.youtube.com/watch?v=0Ees8KFCUXM
[youtube] 0Ees8KFCUXM: Downloading webpage
[youtube] 0Ees8KFCUXM: Downloading ios player API JSON
[youtube] 0Ees8KFCUXM: Downloading andro

...

Error downloading audio for YTID 4v8gc0VhmqY: ERROR: [youtube] 4v8gc0VhmqY: Video unavailable
[youtube] Extracting URL: https://www.youtube.com/watch?v=4xTGouBKGN4
[youtube] 4xTGouBKGN4: Downloading webpage
[youtube] 4xTGouBKGN4: Downloading ios player API JSON
[youtube] 4xTGouBKGN4: Downloading android player API JSON
[youtube] 4xTGouBKGN4: Downloading m3u8 information
[info] 4xTGouBKGN4: Downloading 1 format(s): 251
[download] Destination: Data\Screaming\4xTGouBKGN4.webm
[download] 100% of    5.02MiB in 00:00:00 at 6.24MiB/s   
[ExtractAudio] Destination: Data\Screaming\4xTGouBKGN4.wav
Deleting original file Data\Screaming\4xTGouBKGN4.webm (pass -k to keep)
Downloaded audio for YTID 4xTGouBKGN4
Trimmed audio for YTID 4xTGouBKGN4
[youtube] Extracting URL: https://www.youtube.com/watch?v=4xf8wsy94EU
[youtube] 4xf8wsy94EU: Downloading webpage
[youtube] 4xf8wsy94EU: Downloading ios player API JSON
[youtube] 4xf8wsy94EU: Downloading android player API JSON
[youtube] 4xf8wsy94EU: Downloadi

ERROR: [youtube] 5-CaTkh5xUI: Private video. Sign in if you've been granted access to this video


Error downloading audio for YTID 5-CaTkh5xUI: ERROR: [youtube] 5-CaTkh5xUI: Private video. Sign in if you've been granted access to this video
[youtube] Extracting URL: https://www.youtube.com/watch?v=51YYDbq10A0
[youtube] 51YYDbq10A0: Downloading webpage
[youtube] 51YYDbq10A0: Downloading ios player API JSON
[youtube] 51YYDbq10A0: Downloading android player API JSON


ERROR: [youtube] 51YYDbq10A0: Video unavailable


Error downloading audio for YTID 51YYDbq10A0: ERROR: [youtube] 51YYDbq10A0: Video unavailable
[youtube] Extracting URL: https://www.youtube.com/watch?v=52waxTykuLc
[youtube] 52waxTykuLc: Downloading webpage
[youtube] 52waxTykuLc: Downloading ios player API JSON
[youtube] 52waxTykuLc: Downloading android player API JSON
[youtube] 52waxTykuLc: Downloading m3u8 information
[youtube] 52waxTykuLc: Downloading MPD manifest
[info] 52waxTykuLc: Downloading 1 format(s): 251
[download] Destination: Data\Screaming\52waxTykuLc.webm
[download] 100% of    3.23MiB in 00:00:00 at 3.85MiB/s   
[ExtractAudio] Destination: Data\Screaming\52waxTykuLc.wav
Deleting original file Data\Screaming\52waxTykuLc.webm (pass -k to keep)
Downloaded audio for YTID 52waxTykuLc
Trimmed audio for YTID 52waxTykuLc
[youtube] Extracting URL: https://www.youtube.com/watch?v=54MkoyLle2Q
[youtube] 54MkoyLle2Q: Downloading webpage
[youtube] 54MkoyLle2Q: Downloading ios player API JSON
[youtube] 54MkoyLle2Q: Downloading android 